# Bloch-Torrey Equation

## Introduction

Here we solve the Bloch-Torrey equation on a unit square, with the diffusion coefficient $D(x)$, relaxation rate $R(x)$, and resonance frequency $\omega(x)$ all given as a generic functions.
The strong form of the Bloch-Torrey equation is given by

\begin{align}
    \frac{\partial u_x}{\partial t} &= \nabla \cdot (D \nabla u_x) - R u_x + \omega u_y  \quad x \in \Omega\\
    \frac{\partial u_y}{\partial t} &= \nabla \cdot (D \nabla u_y) - R u_y - \omega u_x  \quad x \in \Omega,
\end{align}

where $\vec{u}=[u_x,u_y]$ is the transverse magnetization, and $\Omega$ the domain.

We will consider homogeneous Neumann boundary conditions such that

\begin{align}
    \nabla \vec{u}(x) \cdot \hat{n} &= \vec{0}  \quad x \in \partial \Omega\\
\end{align}

where $\partial \Omega$ denotes the boundary of $\Omega$, and $\cdot$ is a tensor contraction.

The initial condition is given generically as

\begin{equation}
    \vec{u}(x,t=0) = \vec{u}_0 (x)  \quad x \in \Omega
\end{equation}


The resulting weak form is given by
\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    &= -\int_{\Omega}
    -\vec{v} \cdot \nabla \cdot ( D \, \nabla \vec{u} ) +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega \\
    &= -\int_{\Omega}
    D \, \nabla \vec{v} : \nabla \vec{u} +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega + 
    \int_{\partial\Omega} \vec{v} \cdot (D\nabla\vec{u} \cdot \hat{n}) \, d\Gamma,
\end{align}
where $\vec{v}$ is a suitable test function.

In this notebook, we will assume homogeneous Neumann boundary conditions on all boundaries by taking $D\nabla\vec{u} \cdot \hat{n} = 0$. Therefore, the final weak form is simply
\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    = -\int_{\Omega}
    D \, \nabla \vec{v} : \nabla \vec{u} +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega
\end{align}

Note: in two dimensions, the cross product is simply a scalar. However, `Tensors.jl` defines the two dimensional cross product by first extending the 2D vectors into 3D. Below, we use the symbol $\boxtimes$ to denote the scalar version, which is the same as taking the third component of the vector version

## Commented Program

Now we solve the problem in JuAFEM. What follows is a program spliced with comments.

First we load the JuAFEM package.

In [1]:
# using Distributed
# addprocs(8; restrict = true, enable_threaded_blas = true);
# @everywhere begin
#     include("init.jl")
# end

In [2]:
include("init.jl")

**Pack circles**: Pack circles with a specified packing density $\eta$

In [3]:
btparams = BlochTorreyParameters{Float64}(
    theta = π/2,
    AxonPDensity = 0.7,
    g_ratio = 0.8,
    D_Tissue = 200.0, #0.5, # [μm²/s]
    D_Sheath = 25.0, #0.5, # [μm²/s]
    D_Axon = 200.0, #0.5, # [μm²/s]
    K_perm = 1.0, #0.0 # [μm/s]
);

In [5]:
TE_typical = 10e-3
nDim = 2
dist_typical = (2π * btparams.R_mu)/4
time_typical = TE_typical/2
D_Maximal_Dephasing = dist_typical^2/(2 * nDim * time_typical)

26.105103640881353

In [6]:
Dim = 2;
Ncircles = 20;

η = btparams.AxonPDensity; # goal packing density
ϵ = 0.1 * btparams.R_mu; # overlap occurs when distance between circle edges is ≤ ϵ
α = 1e-1; # covariance penalty weight (enforces circular distribution)
β = 1e-6; # mutual distance penalty weight
λ = 1.0; # overlap penalty weight (or lagrange multiplier for constrained version)

rs = rand(radiidistribution(btparams), Ncircles);
@time initial_circles = GreedyCirclePacking.pack(rs; goaldensity = 1.0, iters = 100)
@show minimum(radius.(initial_circles));
@show estimate_density(initial_circles);

  1.964759 seconds (3.43 M allocations: 169.017 MiB, 5.93% gc time)
minimum(radius.(initial_circles)) = 0.21746941300305322
estimate_density(initial_circles) = 0.8005944845646994


In [7]:
@show minimum_signed_edge_distance(initial_circles);

minimum_signed_edge_distance(initial_circles) = -2.220446049250313e-16


In [8]:
@show estimate_density(initial_circles);

estimate_density(initial_circles) = 0.8005944845646994


In [9]:
@time outer_circles = EnergyCirclePacking.pack(initial_circles;
    autodiff = true,
    secondorder = false,
    setcallback = false,
    goaldensity = η,
    distancescale = btparams.R_mu,
    weights = [α, β, λ],
    epsilon = ϵ
);
inner_circles = scale_shape.(outer_circles, btparams.g_ratio);

  6.406330 seconds (12.96 M allocations: 643.816 MiB, 13.73% gc time)


In [10]:
dmin = minimum_signed_edge_distance(outer_circles)
@show covariance_energy(outer_circles)
@show estimate_density(outer_circles)
@show is_any_overlapping(outer_circles)
@show (dmin, ϵ, dmin > ϵ);

covariance_energy(outer_circles) = 0.3041655559718328
estimate_density(outer_circles) = 0.6999999999999995
is_any_overlapping(outer_circles) = false
(dmin, ϵ, dmin > ϵ) = (0.04164838429702583, 0.046000000000000006, false)


**Generate mesh**: Rectangular mesh with circles possibly only partly contained or completely excluded

In [11]:
h0 = minimum(radius.(outer_circles))*(1-btparams.g_ratio); # fraction of size of minimum torus width
h_min = 1.0*h0; # minimum edge length
h_max = 5.0*h0; # maximum edge length
h_range = 10.0*h0; # distance over which h increases from h_min to h_max
h_rate = 0.6; # rate of increase of h from circle boundaries (power law; smaller = faster radial increase)

0.6

In [5]:
bdry, _ = opt_subdomain(outer_circles);

In [13]:
@time exteriorgrids, torigrids, interiorgrids, parentcircleindices = disjoint_rect_mesh_with_tori(
    bdry, inner_circles, outer_circles, h_min, h_max, h_range, h_rate;
    CIRCLESTALLITERS = 500, EXTERIORSTALLITERS = 500, plotgrids = false, exterior_tiling = (1, 1)
);

1/20: Interior
2/20: Interior
3/20: Interior
4/20: Interior
5/20: Interior
6/20: Interior
7/20: Interior
8/20: Interior
9/20: Interior
10/20: Interior
11/20: Interior
12/20: Interior
13/20: Interior
14/20: Interior
15/20: Interior
16/20: Interior
17/20: Interior
18/20: Interior
19/20: Interior
20/20: Interior
1/20: Annular
2/20: Annular
3/20: Annular
4/20: Annular
5/20: Annular
6/20: Annular
7/20: Annular
8/20: Annular
9/20: Annular
10/20: Annular
11/20: Annular
12/20: Annular
13/20: Annular
14/20: Annular
15/20: Annular
16/20: Annular
17/20: Annular
18/20: Annular
19/20: Annular
20/20: Annular
1/1, 1/1: Exterior
 65.936739 seconds (332.18 M allocations: 10.216 GiB, 15.02% gc time)


In [14]:
sum(area.(exteriorgrids)) + sum(area.(torigrids)) + sum(area.(interiorgrids)) - area(bdry)

-1.7763568394002505e-15

In [15]:
simpplot(vcat(exteriorgrids[:], torigrids, interiorgrids); newfigure = true, axis = mxaxis(bdry));

## Diffusion coefficient $D(x)$, relaxation rate $R(x)$, and resonance frequency $\omega(x)$

These functions are defined within `doassemble!`

### Trial and test functions
A `CellValues` facilitates the process of evaluating values and gradients of
test and trial functions (among other things). Since the problem
is a scalar problem we will use a `CellScalarValues` object. To define
this we need to specify an interpolation space for the shape functions.
We use Lagrange functions (both for interpolating the function and the geometry)
based on the reference "cube". We also define a quadrature rule based on the
same reference cube. We combine the interpolation and the quadrature rule
to a `CellScalarValues` object.

### Degrees of freedom
Next we need to define a `DofHandler`, which will take care of numbering
and distribution of degrees of freedom for our approximated fields.
We create the `DofHandler` and then add a single field called `u`.
Lastly we `close!` the `DofHandler`, it is now that the dofs are distributed
for all the elements.

Now that we have distributed all our dofs we can create our tangent matrix,
using `create_sparsity_pattern`. This function returns a sparse matrix
with the correct elements stored.

We can inspect the pattern using the `spy` function from `UnicodePlots.jl`.
By default the stored values are set to $0$, so we first need to
fill the stored values, e.g. `K.nzval` with something meaningful.

In [16]:
#using UnicodePlots
#fill!(K.nzval, 1.0)
#spy(K; height = 25)

### Boundary conditions
In JuAFEM constraints like Dirichlet boundary conditions are handled by a `ConstraintHandler`. However, here we will have no need to directly enforce boundary conditions, since Neumann boundary conditions have already been applied in the derivation of the weak form.

### Assembling the linear system
Now we have all the pieces needed to assemble the linear system, $K u = f$.
We define a function, `doassemble` to do the assembly, which takes our `cellvalues`,
the sparse matrix and our DofHandler as input arguments. The function returns the
assembled stiffness matrix, and the force vector.

In [6]:
myelinprob = MyelinProblem(btparams);

In [7]:
myelinsubdomains = createmyelindomains(exteriorgrids[:], torigrids, interiorgrids, outer_circles, inner_circles);

In [8]:
# @time map!(m -> doassemble!(m, myelinprob), myelinsubdomains, myelinsubdomains);
@time doassemble!.(myelinsubdomains, Ref(myelinprob));

  4.226366 seconds (30.86 M allocations: 721.460 MiB, 10.17% gc time)


In [9]:
# @time map!(m -> (factorize!(getdomain(m)); return m), myelinsubdomains, myelinsubdomains);
@time factorize!.(getdomain.(myelinsubdomains));

  0.510446 seconds (752.90 k allocations: 50.083 MiB, 7.03% gc time)


In [10]:
@time combinedmyelindomain = MyelinDomain(PermeableInterfaceRegion(), myelinprob, myelinsubdomains);

 20.526573 seconds (192.77 M allocations: 6.714 GiB, 20.61% gc time)


In [11]:
@time factorize!(combinedmyelindomain);

  0.040883 seconds (2.22 k allocations: 11.816 MiB, 35.32% gc time)


In [12]:
myelindomains = [combinedmyelindomain];

In [13]:
# for II in ips #[rand(1:length(ips))]
#     Se = [combinedmyelindomain.domain.K[2i-1,2j-1] for i in II for j in II]
#     Se = Matrix(reshape(6 .* Se, (4,4)))
#     @assert maximum(abs, sum(Se, dims=2)) < 5*eps(Float64)
# end

In [14]:
# for i in II
#     n = getnodes(getgrid(combinedmyelindomain))[i]
#     @show n
# end

In [15]:
# AllNodes = getnodes(getgrid(combinedmyelindomain))
# XMatrix, YMatrix = zeros(2, length(ips)), zeros(2, length(ips))
# for j in 1:length(ips)
#     idx = ips[j]
#     p1, p2 = AllNodes[idx[1]].x, AllNodes[idx[2]].x
#     XMatrix[1,j] = p1[1]
#     YMatrix[1,j] = p1[2]
#     XMatrix[2,j] = p2[1]
#     YMatrix[2,j] = p2[2]
# end

In [16]:
# mxcall(:plot, 0, XMatrix, YMatrix, "bx-")
# mxcall(:axis, 0, "image")

### Plotting the resonance frequency map $\omega(x)$
$\omega(x)$ for each region can be easily created by accessing the `Omega` field of a `BlochTorreyProblem` object. Now, evaluate $\omega(x)$ on each node `x` and plot the resuling field map overtop of the tesselation.

In [17]:
omegavalues = omegamap.(Ref(myelinprob), myelinsubdomains);

In [18]:
simpplot(getgrid.(myelindomains); newfigure = true, axis = mxaxis(bdry), facecol = reduce(vcat, omegavalues));

### Solution of the differential equation system
The last step is to solve the system. First we call `doassemble`
to obtain the global stiffness matrix `K` and force vector `f`.
Then, to account for the boundary conditions, we use the `apply!` function.
This modifies elements in `K` and `f` respectively, such that
we can get the correct solution vector `u` by using `\`.

In [19]:
if !@isdefined AllSols
    global AllSols = Dict()
end;

In [22]:
tspan = (0.0, 320.0e-3);
dt = 10e-3;
ts = tspan[1]:dt/2:tspan[2]
u0 = Vec{2}((0.0, 1.0)); # initial pi/2 pulse

In [23]:
probs = [ODEProblem(m, interpolate(u0, m), tspan; invertmass = true) for m in myelindomains];
sols = Vector{ODESolution}(undef, length(probs));

In [24]:
@time sols = map!(sols, 1:length(probs), probs) do i, prob
    println("i = $i/$(length(sols)): ")
    cb = CPMGCallback(tspan; TE = dt)
    sol = @time solve(prob, ExpokitExpmv(prob.p[1]; m = 30);
        dense = false, # don't save all intermediate time steps
        saveat = ts, # timepoints to save solution at
        tstops = ts, # ensure stopping at all ts points
        dt = dt,
        reltol = 1e-4,
        callback = cb
    )
    return sol
end;

i = 1/1: 
π-pulse at t = 5.0 ms
π-pulse at t = 15.0 ms
π-pulse at t = 25.0 ms
π-pulse at t = 35.0 ms
π-pulse at t = 45.0 ms
π-pulse at t = 55.0 ms
π-pulse at t = 65.0 ms
π-pulse at t = 75.0 ms
π-pulse at t = 85.0 ms
π-pulse at t = 95.0 ms
π-pulse at t = 105.0 ms
π-pulse at t = 115.0 ms
π-pulse at t = 125.0 ms
π-pulse at t = 135.0 ms
π-pulse at t = 145.0 ms
π-pulse at t = 155.0 ms
π-pulse at t = 165.0 ms
π-pulse at t = 175.0 ms
π-pulse at t = 185.0 ms
π-pulse at t = 195.0 ms
π-pulse at t = 205.0 ms
π-pulse at t = 215.0 ms
π-pulse at t = 225.0 ms
π-pulse at t = 235.0 ms
π-pulse at t = 245.0 ms
π-pulse at t = 255.0 ms
π-pulse at t = 265.0 ms
π-pulse at t = 275.0 ms
π-pulse at t = 285.0 ms
π-pulse at t = 295.0 ms
π-pulse at t = 305.0 ms
π-pulse at t = 315.0 ms
1382.517242 seconds (40.96 M allocations: 166.080 GiB, 2.41% gc time)
1390.486047 seconds (56.50 M allocations: 166.884 GiB, 2.46% gc time)


In [25]:
AllSols[:expokit_m30] = deepcopy(sols);

┌ Warning: /home/coopar7/Documents/code/BlochTorreyExperiments-master/Experiments/MyelinWaterOrientation/MWF-utils.jl is not a file, omitting from revision tracking
└ @ Revise /home/coopar7/.julia/packages/Revise/TmjcT/src/parsing.jl:24


In [43]:
probs = [ODEProblem(m, interpolate(u0, m), tspan; invertmass = true) for m in myelindomains];
sols = Vector{ODESolution}(undef, length(probs));

In [44]:
@time sols = map!(sols, 1:length(probs), probs) do i, prob
    println("i = $i/$(length(sols)): ")
    
#     Kfact = lu(prob.p[2])
#     mylinsolve!(::Type{Val{:init}},f,u0) = (x, A, b, update_matrix = false) -> copyto!(x, Kfact\b)
#     alg = TRBDF2(;linsolve = mylinsolve!)
#     alg = ABDF2(;linsolve = mylinsolve!)
#     alg = Rosenbrock23(linsolve = LinSolveFactorize(lu))#;linsolve = mylinsolve!)
    
#     alg = ARKODE(
#         Sundials.Implicit();
#         method = :Functional,
#         linear_solver = :None,
#         order = 4
#     )
    alg = CVODE_BDF(;
        method = :Functional, #method = :Newton,
        linear_solver = :None, #linear_solver = :GMRES,
        max_order = 5
    )
#     alg = CVODE_Adams(;
#         method = :Functional,
#         linear_solver = :None,
#         max_order = 12
#     )
    
    cb = CPMGCallback(tspan; TE = dt)
    sol = @time solve(prob, alg;
        dense = false, # don't save all intermediate time steps
        saveat = ts, # timepoints to save solution at
        tstops = ts, # ensure stopping at all `ts` points
        dt = 0.5 * h0^2/(4*btparams.D_Tissue), # conservative initial dt; timescale for diffusion of distance h0
        reltol = 1e-4,
        abstol = 1e-4,
        callback = cb #callback = CPMGCallback(tspan; TE = dt)
    )
    return sol
end;

i = 1/1: 
π-pulse at t = 5.0 ms



[CVSDLS ERROR]  CVDlsSetJacFn
  Linear solver memory is NULL.



π-pulse at t = 15.0 ms


InterruptException: InterruptException:

In [45]:
AllSols[:cvode_bdf_functional_order5] = deepcopy(sols);

In [46]:
let
    local sols1 = AllSols[:expokit_m30]
#     local sols1 = AllSols[:arkode_gmres_order4]
#     local sols2 = AllSols[:arkode_expl_gmres_order4]
    local sols2 = AllSols[:cvode_bdf_functional_order5]
#     local sols2 = AllSols[:cvode_bdf_gmres_max_order5]
#     local sols2 = AllSols[:cvode_adams_functional_max_order12]
#     local sols2 = AllSols[:rosenbrock23_mylinsolve]
    
    local u0 = reduce(vcat, p.u0 for p in probs)
    local u1 = reduce(vcat, s.u[end] for s in sols1)
    local u2 = reduce(vcat, s.u[end] for s in sols2)
    
    local S1 = norm.(calcsignal(myelindomains, sols1, tspan[1]:dt:tspan[2]))
    local S2 = norm.(calcsignal(myelindomains, sols2, tspan[1]:dt:tspan[2]))
    
    @show norm(u1.-u2)/norm(u1)
    @show maximum(abs, u1.-u2)/maximum(abs, u1)
    @show norm(S1.-S2)/norm(S1)
    @show maximum(abs, S1.-S2)/maximum(abs, S1)
    @show norm(u0)
    @show norm(u1), norm(u2)
    @show norm(S1), norm(S2)
    @show maximum(abs, u1), maximum(abs, u2)
    @show maximum(abs, S1), maximum(abs, S2)
end;

UndefRefError: UndefRefError: access to undefined reference

In [26]:
sols = deepcopy(AllSols[:expokit_m30]);
# sols = deepcopy(AllSols[:cvode_bdf_gmres_max_order5]);
# sols = deepcopy(AllSols[:cvode_bdf_functional_order5]);

In [27]:
Umagn = reduce(vcat, norm.(reinterpret(Vec{2,Float64}, s.u[end])) for s in sols);
# Uphase = reduce(vcat, angle.(reinterpret(Vec{2,Float64}, s.u[end])) for s in sols);
simpplot(getgrid.(myelindomains); newfigure = true, axis = mxaxis(bdry), facecol = Umagn); mxcall(:title, 0, "Magnitude")
# simpplot(getgrid.(myelindomains); newfigure = true, axis = mxaxis(bdry), facecol = Uphase); mxcall(:title, 0, "Phase")

In [28]:
# mxcall(:caxis,0,[0.0 5e-5])

In [29]:
Signals = map(myelindomains, sols) do m, s
    [integrate(s(t), m) for t in tspan[1]:dt:tspan[2]]
end;

In [30]:
Stotal = sum(Signals);

In [31]:
Tspan = tspan[2] - tspan[1]
@show btparams.R2_lp;
@show (-1/Tspan)*log(norm(Stotal[end])/norm(Stotal[1]));
@show exp(-tspan[end]*btparams.R2_lp);
@show norm(Stotal[end])/norm(Stotal[1]);

btparams.R2_lp = 15.873015873015873
(-1 / Tspan) * log(norm(Stotal[end]) / norm(Stotal[1])) = 21.1225713900671
exp(-(tspan[end]) * btparams.R2_lp) = 0.006223859418487457
norm(Stotal[end]) / norm(Stotal[1]) = 0.0011601304250208162


In [32]:
exact_mwf = getmwf(outer_circles, inner_circles, bdry);
@show exact_mwf;

exact_mwf = 0.2654265963783337


In [33]:
mxcall(:addpath, 0, realpath("MyelinWaterImaging/MATLAB/"));

In [34]:
MWImaps, MWIdist, MWIpart = fitmwfmodel(Stotal, NNLSRegression();
    T2Range = [8e-3, 2.0],
    spwin = [8e-3, 24.75e-3],
    mpwin = [25.25e-3, 200e-3],
    nT2 = 32,
    RefConAngle = 165.0,
    PLOTDIST = true
);
getmwf(NNLSRegression(), MWImaps, MWIdist, MWIpart)

0.1313297834801338

In [35]:
# TestStotalMagn = mwimodel(TwoPoolMagnToMagn(), tspan[1]:dt:tspan[2], modelfit.param)
# TestStotal = [Vec{2}((zero(y),y)) for y in TestStotalMagn];

In [36]:
for modeltype in (TwoPoolMagnToMagn(), ThreePoolMagnToMagn(), ThreePoolCplxToMagn(), ThreePoolCplxToCplx())
    local modelfit, errors, mwf
    println("Model: $modeltype"); flush(stdout)
    modelfit, errors = fitmwfmodel(Stotal, modeltype; TE = dt);
    mwf = getmwf(modeltype, modelfit, errors)
    println("mwf: $mwf"); flush(stdout)
    errors == nothing ? display(modelfit.param) : display([modelfit.param errors]); flush(stdout)
end

Model: TwoPoolMagnToMagn()
mwf: 0.13546135589421932


4×2 Array{Float64,2}:
  1.48222  0.0150375
  9.45979  0.0197345
 81.1729   1.82982  
 20.8114   0.0236878

Model: ThreePoolMagnToMagn()
mwf: 0.12611141889516014


6×2 Array{Float64,2}:
  1.38526  0.00486312
  6.49315  0.797022  
  3.10597  0.791486  
 89.1452   0.308108  
 19.8989   0.18648   
 23.6569   0.621536  

Model: ThreePoolCplxToMagn()
mwf: 0.16725931274078285


8×2 Array{Float64,2}:
  1.82453    1.32016
  6.01359   14.2762 
  3.07027   15.4918 
 71.1816    14.0873 
 18.3061     8.61642
 22.7963     5.31249
 -1.07138  179.801  
 -1.47969    2.69874

Model: ThreePoolCplxToCplx()
mwf: 0.11544506549622016


10×2 Array{Float64,2}:
  1.26809   0.223242  
  8.30479   3.47577   
  1.41145   3.30041   
 92.8616   14.6852    
 20.2422    1.18528   
 28.0343   15.423     
 46.9226    2.41192   
 50.0046    0.0683543 
 49.5182    1.10129   
 -1.55575   0.00968076

In [37]:
widths(bdry)

2-element Tensor{1,2,Float64,2}:
 3.3135629877727597
 3.3135629877727597

In [38]:
@show getmwf(outer_circles, inner_circles, bdry);
@show getmwf(Stotal, TwoPoolMagnToMagn(); TE = dt, fitmethod = :local);
@show getmwf(Stotal, ThreePoolMagnToMagn(); TE = dt, fitmethod = :local);
@show getmwf(Stotal, ThreePoolCplxToMagn(); TE = dt, fitmethod = :local);
@show getmwf(Stotal, ThreePoolCplxToCplx(); TE = dt, fitmethod = :local);

getmwf(outer_circles, inner_circles, bdry) = 0.2654265963783337
getmwf(Stotal, TwoPoolMagnToMagn(); TE=dt, fitmethod=:local) = 0.13546135589421932
getmwf(Stotal, ThreePoolMagnToMagn(); TE=dt, fitmethod=:local) = 0.12611141889516014
getmwf(Stotal, ThreePoolCplxToMagn(); TE=dt, fitmethod=:local) = 0.16725931274078285
getmwf(Stotal, ThreePoolCplxToCplx(); TE=dt, fitmethod=:local) = 0.11544506549622016


In [39]:
p0 = initialparams(ThreePoolCplxToCplx(), ts, Stotal)[1];
modelfit, errors = fitmwfmodel(Stotal, ThreePoolCplxToCplx(); TE = dt);

In [40]:
# mwimodel(ThreePoolCplxToCplx(), ts, modelfit.param);
# [mwimodel(ThreePoolCplxToCplx(), ts,  modelfit.param) |> x -> reinterpret(ComplexF64, x) complex.(Stotal)]
# [mwimodel(ThreePoolCplxToCplx(), ts, p0) |> x -> reinterpret(ComplexF64, x) complex.(Stotal)]

In [53]:
myelin_area = intersect_area(outer_circles, bdry) - intersect_area(inner_circles, bdry)
total_area = area(bdry)
y_biexp = @. (total_area - myelin_area) * exp(-(tspan[1]:dt:tspan[2])*btparams.R2_lp) + myelin_area * exp(-(tspan[1]:dt:tspan[2])*btparams.R2_sp);

In [57]:
mxcall(:figure, 0)
mxcall(:plot, 0, collect(1000.0.*(tspan[1]:dt:tspan[2])), [norm.(Stotal) y_biexp])

In [58]:
mxcall(:legend, 0, "Simulated", "Bi-Exponential")
mxcall(:title, 0, "Signal Magnitude vs. Time")
mxcall(:xlabel, 0, "Time [ms]"); mxcall(:xlim, 0, 1000.0 .* [tspan...])
mxcall(:ylabel, 0, "S(t) Magnitude")

In [ ]:
mxcall(:ylim, 0, [0.0 17.5303])

In [44]:
# mxcall(:figure, 0)
# mxcall(:plot, 0, collect(1000.0.*ts), reduce(hcat, map(S->norm.(S), Signals)))

In [45]:
# prob = ODEProblem((du,u,p,t)->A_mul_B!(du,p[1],u), u0, tspan, (Amap,));

In [46]:
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=100); # penelope: 17.42s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=50); # penelope: 30.09s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=10); # penelope: 103.5s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-8, norm=expmv_norm); # penelope: 53.2s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-6, norm=expmv_norm); # penelope: 44.4s

In [47]:
#@time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-8, alg_hints=:stiff); # penelope: 90.21s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-4, alg_hints=:stiff); # penelope: 33.44s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:BCG); saveat=tspan, reltol=1e-4, alg_hints=:stiff) # penelope: 53.66s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:TFQMR); saveat=tspan, reltol=1e-4, alg_hints=:stiff) # penelope: 18.99s but low accuracy

In [48]:
#prob_Ku = ODEProblem(K_mul_u!, u0, tspan, (K,), mass_matrix=M);
#@time sol_Ku = solve(prob_Ku, Rosenbrock23(), saveat=tspan, reltol=1e-4, alg_hints=:stiff) #DNF
#@time sol_Ku = solve(prob_Ku, Rodas4(), saveat=tspan, reltol=1e-4, alg_hints=:stiff) #DNF

In [49]:
# @show norm(sol.u[end] - u)/maximum(abs,u);
# @show maximum(sol.u[end] - u)/maximum(abs,u);

# Testing

In [62]:
# using Test
# using DiffEqCallbacks
# using Plots
# gr(); default(fmt = :png)

In [63]:
# let
#     local u0 = [1.,0.]
#     local function fun2(du, u, p, t)
#        du[2] = -u[1]
#        du[1] = u[2]
#     end
#     local tspan = (0.0,10.0)
#     local prob = ODEProblem(fun2,u0,tspan)

# #     function condition2(u,t,integrator)
# #       get_du(integrator)[1]>0
# #     end
# #     affect2!(integrator) = terminate!(integrator)
# #     local cb = DiscreteCallback(condition2,affect2!)

# #     function make_time_choice()
# #         ts = collect(tspan[1]+pi/2:pi/2:tspan[2])
# #         time_choice(integrator) = isempty(ts) ? typemax(eltype(ts)) : popfirst!(ts)
# #         return time_choice
# #     end
# #     function user_affect!(integrator)
# #         println("π-pulse at t = $(integrator.t)")
# #         return @views(integrator.u[2:2:end] .= -integrator.u[2:2:end])
# #     end
# #     local cb = IterativeCallback(make_time_choice(), user_affect!, eltype(tspan); initial_affect = false)
# # #     local cb = PeriodicCallback(user_affect!, 2.0; initial_affect = false)

#     local cb = CPMGCallback(tspan; TE = 2.0)
# #     local cb = CPMGCallback(tspan; pulsetimes = [0.0])

# #     local sol = solve(prob, Rosenbrock23(); callback = cb);
# #     local sol = solve(prob, CVODE_BDF(); callback = cb);
#     local sol = solve(prob, CVODE_Adams(); callback = cb);
# #     local sol = solve(prob, ARKODE(); callback = cb);

#     # @test sol.t[end] < 3.5
#     plot(sol)
# end


In [56]:
# using BlochTorreyUtilsTest

### Single Axon

In [57]:
# BlochTorreyUtilsTest.singleaxontests(
#     BlochTorreyParameters{Float64}(
#         ChiI = -60e-9,
#         ChiA = -120e-9
#     );
#     PLOTOMEGA = false
# );

### Multiple Axons

In [58]:
# domainsetup = BlochTorreyUtilsTest.multipleaxons();

In [59]:
# BlochTorreyUtilsTest.multipleaxontests(
#     BlochTorreyParameters{Float64}(
#         ChiI = -60e-9,
#         ChiA = -120e-9
#     ),
#     domainsetup;
#     PLOTOMEGA = false
# );

### Exporting to VTK
To visualize the result we export the grid and our field `u`
to a VTK-file, which can be viewed in e.g. [ParaView](https://www.paraview.org/).

In [60]:
# vtk_grid("bloch_torrey_equation", dh) do vtk
#     vtk_point_data(vtk, dh, u)
# end